In [ ]:
import os
import re
import ast
import torch
import json
import huggingface_hub

from tqdm import tqdm
from openai import OpenAI

## LLM 정의 및 프롬프트 hub
from langchain import hub
from langchain_openai import ChatOpenAI

## Function call
from langchain.tools.retriever import create_retriever_tool

## jsonl 파일 로더
from langchain.document_loaders import JSONLoader

## 인코더, 벡터 DB
from langchain.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

## Huggingface와 Langchain 연동
from langchain_huggingface.llms import HuggingFacePipeline
from langchain_huggingface.embeddings import HuggingFaceEmbeddings

from langchain.tools import Tool
from langchain.agents import initialize_agent, AgentType


from transformers import AutoModelForCausalLM, AutoTokenizer

from dotenv import load_dotenv
load_dotenv("../keys.env")

openai_api_key = os.getenv('OPENAI_API_KEY')
os.environ['OPENAI_API_KEY'] = openai_api_key

hf_token = os.getenv("HF_TOKEN")
huggingface_hub.login(hf_token)

## documents.jsonl 로드 및 처리

1. 문서 데이터를 encoder 모델로 임베딩.
2. Faiss를 벡터 DB로 정하고 벡터를 저장.

In [ ]:
loader = JSONLoader(
    file_path='../dataset/documents.jsonl', 
    json_lines=True, 
    jq_schema='.content'  # 'content' 필드만 추출
)
documents = loader.load()
print(len(documents))

print(documents[0])

In [ ]:
## OpenAIEmbeddings로 임베딩
# embedding_model = OpenAIEmbeddings()
# embeddings = embedding_model.embed_documents([doc.page_content for doc in documents])

## 벡터를 FAISS에 저장
# faiss_index = FAISS.from_texts([doc.page_content for doc in documents], embedding_model)

## Hugging Face Embeddings 설정
model_name = "jhgan/ko-sroberta-multitask"
model_kwargs = {"device": "cuda:0"}
encode_kwargs = {"normalize_embeddings": False}

embedding_model = HuggingFaceEmbeddings(model_name=model_name, model_kwargs=model_kwargs, encode_kwargs=encode_kwargs)

## 문서 내용 가져오기
contents = [doc.page_content for doc in documents]

## 벡터 DB 생성 및 저장
vector_db = FAISS.from_texts(texts=contents, embedding=embedding_model)

## 검색기 설정
retriever = vector_db.as_retriever(search_kwargs={"k": 10})

## 벡터 DB에 저장된 벡터의 수 확인
print(f"벡터 DB에 저장된 벡터의 수: {vector_db.index.ntotal}")
print(f"입력한 텍스트의 수: {len(contents)}")

In [ ]:
retriever.invoke("에너지 균형을 유지하는 방법은 무엇인가요?")

In [ ]:
retriever.invoke("파이썬")

## eval.jsonl 로드 및 처리

```eval.jsonl``` 파일 로드

In [ ]:
def load_jsonl(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        return [json.loads(line) for line in f]

eval_data = load_jsonl("../dataset/eval.jsonl")
print(len(eval_data))
print(eval_data[0].keys())

In [7]:
def convert_to_dialogue_format(messages):
    dialogue = ""
    for msg in messages:
        role = msg['role']
        content = msg['content']
        if role == 'user':
            dialogue += f"user: {content}\n"
        elif role == 'assistant':
            dialogue += f"assistant: {content}\n"
    return dialogue

In [ ]:
sample1 = eval_data[0]['msg']
print(sample1)

dialogue1 = convert_to_dialogue_format(sample1)
print(dialogue1)

In [ ]:
sample2 = eval_data[8]['msg']
print(sample2)

dialogue2 = convert_to_dialogue_format(sample2)
print(dialogue2)

In [ ]:
sample3 = eval_data[21]['msg']
print(sample3)

dialogue3 = convert_to_dialogue_format(sample3)
print(dialogue3)

## LLM agent

1. LLama3 모델, 토크나이저 로드
2. Fucntion Call 정의

상황별로 내가 정해주는게 아니라 LLM이 알아서, 스스로 판단하는 것이 핵심.

다음과 같은 처리 파이프라인을 정의하고자 함.
1. 질의가 멀티턴(multi-turn) 대화인 경우 LLM이 정리해서 Standalone Query를 생성하고 멀티턴이 아닌 경우 건너뛴다.
2. 질의가 ```과학상식```에 해당한다면 encoder로 전달해 임베딩하고 벡터 db로 전달해 유사도 기반 검색을 수행한다. 그렇지 않은 경우 "답변할 수 없습니다."로 처리한다.

Reference

- [https://python.langchain.com/docs/integrations/chat/](https://python.langchain.com/docs/integrations/chat/)
- [https://github.com/sionic-ai/xionic-ko-llama-3-70b](https://github.com/sionic-ai/xionic-ko-llama-3-70b)

In [9]:
llm = ChatOpenAI(base_url="http://sionic.chat:8001/v1",
                 api_key="934c4bbc-c384-4bea-af82-1450d7f8128d",
                 model="xionic-ko-llama-3-70b",
                 temperature=0.2)

In [ ]:
prompt = hub.pull("hwchase17/react-chat-json")

In [11]:
def create_standalone_query(query):
    if isinstance(query, str) and "user:" in query and "assistant:" in query:
        conversation_lines = query.split("\n")
        
        # 대화의 전체 맥락을 활용하여 standalone query 생성
        standalone_query = " ".join([line.strip() for line in conversation_lines if line])

        return {
            "query": standalone_query,
            "multi-turn": True
        }
    else:
        return {
            "query": query.strip(),
            "multi-turn": False
        }


In [ ]:
# Tool definition
standalone_query_tool = Tool(
    name="Standalone Query Generator",
    func=create_standalone_query,
    description="This tool processes dialogue history and generates a standalone query. \
                 For single-turn dialogues, it returns 'multi-turn: False' and the query unchanged as 'query: query'. \
                 For multi-turn dialogues, the tool extracts the user's last question and generates a concise standalone query based on the context of previous interactions. \
                 It returns 'query: standalone_query' and 'multi-turn: True'. \
                 The purpose is to enable the LLM to efficiently summarize dialogue history and generate actionable queries for further processing."
)



# 에이전트 초기화
tools = [standalone_query_tool]
agent = initialize_agent(
    tools=tools,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    llm=ChatOpenAI(),
    verbose=True
)

In [ ]:
output_single_turn = agent.invoke(dialogue1)
print(output_single_turn)

In [ ]:
output_multi_turn = agent.invoke(dialogue2)
print(output_multi_turn)

In [35]:
client = OpenAI()
model = "gpt-4o"

In [36]:
def clean_json_response(response):
    # 코드 블록(예: ```json, ```) 제거
    cleaned_response = re.sub(r'```(?:json)?', '', response).strip()
    return cleaned_response

In [37]:
standalone_content = ("입력된 내용이 한 줄의 문장인지 여러 줄의 대화 내용인지 분류하세요. 반드시 JSON 형식으로 응답하며, 키와 값은 모두 이중 따옴표로 감쌉니다."
                      "- 단일 문장인 경우 : {\"multi_turn\": false, \"query\": \"입력 문장\"} "
                      "- 여러 줄의 대화 내용인 경우 : {\"multi_turn\": true, \"query\": \"대화 내용을 종합하여 만든 새로운 질문\"}")


def create_standalone_query(query):
    completion = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": standalone_content},
            {"role": "user", "content": query}
        ],
    )
    
    response = completion.choices[0].message.content
    cleaned_response = clean_json_response(response)
    
    try:
        json_response = json.loads(cleaned_response)
    except json.JSONDecodeError:
        return {"error": "Invalid JSON response", "response": response}
    
    return json_response

In [38]:
domain_content = (
    "입력된 텍스트가 상식에 대한 질문인지 아니면 단순한 일상적인 대화인지 구분하세요."
    " - 상식에 대한 질문이란, 사용자가 지식이나 정보를 얻기 위해 하는 질문입니다. 예: '나무의 분류에 대해 조사하는 방법은?', 'Dmitri Ivanovsky가 누구야?', '남녀 관계에서 정서적인 행동이 왜 중요해?'"
    " - 일상적인 대화란, 주로 감정이나 의견을 표현하거나 대화의 흐름을 유지하기 위한 내용입니다. 예: '요새 너무 힘들다.', '니가 대답을 잘해줘서 너무 신나!', '이제 그만 얘기해!', '오늘 너무 즐거웠다!', ''너는 누구니?', '너는 어떤 능력을 가지고 있니?"
    " 반드시 JSON 형식으로 응답하세요. 키와 값은 모두 이중 따옴표로 감싸야 합니다."
    " - 상식에 대한 질문인 경우: {\"out_of_domain\": false, \"query\": \"입력된 쿼리를 그대로 반환\"}"
    " - 일상적인 대화인 경우: {\"out_of_domain\": true, \"query\": \"적절한 응답을 할 수 없습니다.\"}"
)


def domain_check(query):
    completion = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": domain_content},
            {"role": "user", "content": query}
        ],
    )
    
    response = completion.choices[0].message.content
    cleaned_response = clean_json_response(response)
    
    try:
        json_response = json.loads(cleaned_response)
    except json.JSONDecodeError:
        return {"error": "Invalid JSON response", "response": response}
    
    return json_response

In [39]:
result1 = create_standalone_query(dialogue1)
print(result1)

print(domain_check(result1['query']))

{'multi_turn': False, 'query': '나무의 분류에 대해 조사해 보기 위한 방법은?'}
{'out_of_domain': False, 'query': '나무의 분류에 대해 조사해 보기 위한 방법은?'}


In [40]:
result2 = create_standalone_query(dialogue2)
print(result2)

print(domain_check(result2['query']))

{'multi_turn': True, 'query': '이란-콘트라 사건이 미국 정치에 미친 영향은 무엇인가요?'}
{'out_of_domain': False, 'query': '이란-콘트라 사건이 미국 정치에 미친 영향은 무엇인가요?'}


In [41]:
result3 = create_standalone_query(dialogue3)
print(result3)

print(domain_check(result3['query']))

{'multi_turn': False, 'query': '요새 너무 힘들다.'}
{'out_of_domain': True, 'query': '적절한 응답을 할 수 없습니다.'}


In [42]:
arr = []
for data in eval_data:
    dialogue = convert_to_dialogue_format(data['msg'])

    interm_query = create_standalone_query(dialogue)    
    final_query = domain_check(interm_query['query'])
    print(f"eval_id : {data['eval_id']}\norg : {dialogue}\nstd_query : {interm_query}\nfinal_query : {final_query}\n")
    
    if final_query['out_of_domain']:
        print("*" * 20)
        print(f"response : {final_query['query']}")
        print("*" * 20)
        arr.append({'eval_id' : data['eval_id'], 'msg' : dialogue, 'final_query' : final_query})

    print()

eval_id : 78
org : user: 나무의 분류에 대해 조사해 보기 위한 방법은?

std_query : {'multi_turn': False, 'query': '나무의 분류에 대해 조사해 보기 위한 방법은?'}
final_query : {'out_of_domain': False, 'query': '나무의 분류에 대해 조사해 보기 위한 방법은?'}


eval_id : 213
org : user: 각 나라에서의 공교육 지출 현황에 대해 알려줘.

std_query : {'multi_turn': False, 'query': '각 나라에서의 공교육 지출 현황에 대해 알려줘.'}
final_query : {'out_of_domain': False, 'query': '각 나라에서의 공교육 지출 현황에 대해 알려줘.'}


eval_id : 107
org : user: 기억 상실증 걸리면 너무 무섭겠다.
assistant: 네 맞습니다.
user: 어떤 원인 때문에 발생하는지 궁금해.

std_query : {'multi_turn': True, 'query': '기억 상실증은 어떤 원인 때문에 발생하나요?'}
final_query : {'out_of_domain': False, 'query': '기억 상실증은 어떤 원인 때문에 발생하나요?'}


eval_id : 81
org : user: 통학 버스의 가치에 대해 말해줘.

std_query : {'multi_turn': False, 'query': '통학 버스의 가치에 대해 말해줘.'}
final_query : {'out_of_domain': False, 'query': '통학 버스의 가치에 대해 말해줘.'}


eval_id : 280
org : user: Dmitri Ivanovsky가 누구야?

std_query : {'multi_turn': False, 'query': 'Dmitri Ivanovsky가 누구야?'}
final_query : {'out_of_domain': False, 'query': 

In [43]:
print(len(arr))

for a in arr:
    print(a['eval_id'])

20
276
261
283
32
94
90
220
245
229
247
67
57
2
227
301
222
83
64
103
218
